In [6]:
from glob import glob
import json
import pandas as pd

downloadeds = (glob("../storage/data/raw/images/*"))
downloadeds = set([d[26:] for d in downloadeds])

In [5]:
len((glob("../storage/data/raw/images/*")))

0

In [8]:
df = pd.read_parquet("storage/data/prepared/train.parquet")

FileNotFoundError: [Errno 2] No such file or directory: 'storage/data/prepared/train.parquet'

In [3]:
import io
from datetime import datetime
from typing import Optional, List
from glob import glob

import requests
from urllib3.util.retry import Retry

from PIL import Image


class ImageDownloader():
    def __init__(self):
        self.failed_downloads = []
        
    def download_save_image(self, row):
        try:
            req = requests.get(f"https://api.divar.ir/support/archivedpost/{row['token']}", timeout=3, headers=headers)
        except:
            return
        for i, image_name in enumerate(row['images']):
            if image_name in downloadeds:
                continue
            if req.status_code == 200:
                archived_at = json.loads(req.content)['archived_post']['archived_at'][:10]
                image_url = self._prepare_post_image_url(row['token'], archived_at, i)
            elif req.status_code == 404:
                image_url = self._prepare_image_url(row['token'], i)
            else:
                print(row['token'], req)
                continue
            
            img = self._download_image(image_url)
            if img:
                img.save("../storage/data/raw/images" + image_name)
            else:
                self.failed_downloads.append(image_name)

    def _download_image(self, image_url) -> Image:
        try:
            req = requests.get(image_url, timeout=3)
            if req.status_code not in [200, 201, 204]:
                logger.error("Download image Error", extra={
                    "event": {
                        "action": "duplicate_image_bot",
                        "method": "download_image",
                        "err": str(image_url)
                    }
                })
                return None
            image_data = req.content
            img = Image.open(io.BytesIO(image_data))
        except Exception as err:
            return None
        return img

    @staticmethod
    def _prepare_image_url(token, i):
        ongoing_image_host_url = "https://s100.divarcdn.com/static/manage_pictures/"
        image_url = ongoing_image_host_url + f"{token}"
        if i:
            image_url += f".{i}.jpg"
        else:
            image_url += ".jpg"        
        return image_url
    @staticmethod
    def _prepare_post_image_url(token, date, i):
        ongoing_image_host_url = "https://divar.ir/archived_img/"
        image_url = ongoing_image_host_url + f"{date}/{token}"
        if i:
            image_url += f".{i}.jpg"
        else:
            image_url += ".jpg"
            
        return image_url
        
import threading
import queue


class Downloader(threading.Thread):
    def __init__(self, df):
        self.df = df
        self.downloader = ImageDownloader()
        super(Downloader, self).__init__()

    def run(self):
        self._get_pictures()

    def _get_pictures(self):
        for i, row in self.df.iterrows():
            self.downloader.download_save_image(row)


In [4]:
#Remember to delete deflate and br from Accept-Encoding

headers = """Host: api.divar.ir
User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0
Accept: application/json, text/plain, */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip
Origin: https://support-admin.divar.ir
DNT: 1
Connection: keep-alive
Referer: https://support-admin.divar.ir/
Cookie: did=30f62811-862f-49c8-bfc0-cd9db3b5b7eb; multi-city=tehran%7C; city=tehran; EOAuthRedirectBack=/archives; sessionid=qe3c6trzkhp9t94v5wh26ca1qged2y2v; EOAuthToken=qFjcSbdQJPY8y+urmTk0vVHPAad4Gu2WOXDWvbnRSZ8=; EOAuthUserInfo=5uhv8htn+alzMHUYzHMmt9TZmh3hM+Dk0N4DoCEG8Fl13fRfNoRWzxWeGdDCewaFhfePpzXty763mLsTFKGvvQ==
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-site
Sec-GPC: 1
Pragma: no-cache
Cache-Control: no-cache"""

headers += "\nAccept: application/json"

headers = headers.split("\n")
headers = [h.split(':') for h in headers]
headers = {h[0].strip():h[1].strip() for h in headers}

In [13]:
from time import time
max_threads = 15
active_threads = 0
picture_threads = []
thread_images = (len(images_url)//max_threads)
download_threads = []

tic = time()
for i in range(max_threads):
    thread_images_url = images_url.loc[i*thread_images:min([len(images_url), (i+1)*thread_images])]
    download_threads.append(Downloader(thread_images_url))
    download_threads[i].start()        

# wait for threads to finish
for picture_thread in picture_threads:
    picture_thread.join()
    print("DONE")
toc = time()

QYwK9Q9W <Response [429]>
QYwK9Q9W <Response [429]>
QYwK9Q9W <Response [429]>
QYwK9Q9W <Response [429]>
QYwK9Q9W <Response [429]>
QYGrf-lf <Response [429]>
QYGrf-lf <Response [429]>
QYGrf-lf QYWLDgcD <Response [429]>
QYA_jmmJ <Response [429]>
QYA_jmmJ <Response [429]>
QYA_jmmJ QYcH1Tvf <Response [429]>
QYsyJCoM <Response [429]>
<Response [429]>
QYA_jmmJ <Response [429]>
QYA_jmmJ QYePNNLG <Response [429]>
QYePNNLG <Response [429]>
QYePNNLG <Response [429]><Response [429]>
QYGrf-lf <Response [429]>
QYGrf-lf <Response [429]><Response [429]>

QYePNNLG <Response [429]>

QYGrf-lf <Response [429]>
QYGrf-lf <Response [429]>
QYB_Nq1e <Response [429]>
QYB_Nq1e <Response [429]>
QYE3tPrM QYaH7nEv <Response [429]>
<Response [429]>
QYE3tPrM <Response [429]>
QYE3tPrM QYp2_oDn <Response [429]>
<Response [429]>
QYE3tPrM <Response [429]>
QYp2_oDn <Response [429]>
QYi3ZxLT <Response [429]>
QYRDddwe QYkH0LT0 <Response [429]>
QYkH0LT0 <Response [429]>
QYkH0LT0 <Response [429]>
<Response [429]>
QYAHz1Ww <Re

QYBrNWhK <Response [429]>
QYnOJ-dv <Response [429]>
QYnOJ-dv <Response [429]>
QYnOJ-dv <Response [429]>
QYnOJ-dv <Response [429]>QYjSIX4A <Response [429]>
QY_eR4QK <Response [429]>
QY_eR4QK <Response [429]>

QYnOJ-dv <Response [429]>
QYfe3Ib_ <Response [429]>
QYfe3Ib_ <Response [429]>
QYfe3Ib_ <Response [429]>
QYIbcmm9 <Response [429]>
QYIbcmm9 <Response [429]>
QYGfqhEN <Response [429]>
QYGfqhEN <Response [429]>
QYGfqhEN <Response [429]>
QYaLNxLH <Response [429]>
QYaLNxLH <Response [429]>
QYaLNxLH <Response [429]>
QYlW9awQQYI3avFz <Response [429]>
QYI3avFz <Response [429]>
QYI3avFz <Response [429]>
QYG3rgYB <Response [429]>
 <Response [429]>
QYlW9awQ <Response [429]>
QYlW9awQ <Response [429]>
QYlW9awQ <Response [429]>
QYlW9awQ <Response [429]>
QYlW9awQ <Response [429]>
QYlW9awQ <Response [429]>
QYlW9awQ <Response [429]>
QYlW9awQ <Response [429]>
QYkXatbU <Response [429]>
QYQSxVYc <Response [429]>
QYQSxVYc <Response [429]>
QYueQis8 <Response [429]>
QYueQis8 <Response [429]>
QYueQis8 <Re

QYXDetjs <Response [429]>
QYXDetjs <Response [429]>
QYXDetjs <Response [429]>
QYFUHwNx <Response [429]>
QYFUHwNx <Response [429]>
QYFUHwNx <Response [429]>
QYvK0LFD <Response [429]>
QYvK0LFD <Response [429]>
QYvK0LFD <Response [429]>QYWzkU0s <Response [429]>
QYWzkU0s <Response [429]>
QYWzkU0s <Response [429]>
QY_qU-s1 <Response [429]>
QY_qU-s1 <Response [429]>
QY_qU-s1 <Response [429]>
QYzWRi-t
QY_qU-s1 <Response [429]>
QYWzkU0s <Response [429]>
QY7WKUwa <Response [429]>
 <Response [429]>
QYzWRi-t <Response [429]>
QYlXK9hw <Response [429]>
QYvK0LFD <Response [429]>
QYvK0LFD <Response [429]>
QYvK0LFD <Response [429]>
QYvK0LFD <Response [429]>
QYvK0LFD <Response [429]>
QYvK0LFD <Response [429]>
QYuv89zl <Response [429]>
QYuv89zl <Response [429]>
QYVz1aJ1 <Response [429]>
QYVz1aJ1 <Response [429]>
QYGfqhEN <Response [429]>
QYGfqhEN <Response [429]>
QYGfqhEN <Response [429]>
QY_-IKkx <Response [429]>
QY_-IKkx <Response [429]>
QY_-IKkx <Response [429]>
QYueFqNg <Response [429]>
QY9OuQDh <Re

QYV_GZa0 <Response [429]>
QYV_GZa0 <Response [429]>
QYpeOHTV <Response [429]>
QYpeOHTV QYG20CHm <Response [429]>
QYfiSUch <Response [429]>
QYujryyM <Response [429]>QY2GOiNy <Response [429]>
<Response [429]>QYfiSUch <Response [429]>

QYujryyM <Response [429]>
QYfiSUch <Response [429]>

QYpeOHTV <Response [429]>
QYmT89Qo <Response [429]>
QYmT89Qo <Response [429]>
QYmT89QoQYtT2VAd <Response [429]>
 <Response [429]>
QYmT89Qo <Response [429]>
QYmT89Qo <Response [429]>
QYmT89Qo <Response [429]>
QYmT89Qo <Response [429]>
QYmT89Qo <Response [429]>
QYmT89Qo <Response [429]>
QYFb6ol2 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Response [429]>
QYaPH7O0 <Re